In [2]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from collections import Counter

# Configuración de directorios
BASE_DIR = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final'
IMAGES_DIR = os.path.join(BASE_DIR, 'imagenes_descargadas')
csv_path = os.path.join(BASE_DIR, 'publicaciones_pixabay_con_rutas.csv')

# Verificar que existe el CSV con rutas
if not os.path.exists(csv_path):
    print("❌ El archivo CSV con rutas no existe. Ejecuta primero el notebook principal.")
else:
    print("✅ CSV encontrado. Cargando dataset...")
    df = pd.read_csv(csv_path)
    print(f"📊 Dataset cargado: {len(df)} imágenes")
    
    # Mostrar primeras filas
    print("\n📋 Primeras 3 imágenes:")
    print(df[['ID_Publicacion', 'Imagen', 'Tematica']].head(3))

✅ CSV encontrado. Cargando dataset...
📊 Dataset cargado: 200 imágenes

📋 Primeras 3 imágenes:
  ID_Publicacion        Imagen Tematica
0        POST_17   POST_17.jpg   comida
1       POST_134  POST_134.jpg    salud
2        POST_14   POST_14.jpg   comida


In [3]:
# Análisis simplificado sin TensorFlow (debido a problemas de compatibilidad)
# Usaremos PIL, OpenCV y scikit-learn para análisis básico

import cv2
from PIL import Image, ImageStat
from sklearn.cluster import KMeans
import colorsys

print("🤖 Configurando análisis de imágenes...")
print("📦 Librerías disponibles:")
print(f"  ✅ OpenCV: {cv2.__version__}")
print(f"  ✅ PIL/Pillow")
print(f"  ✅ scikit-learn para clustering")

# Verificar que tenemos acceso a las imágenes
sample_image = df.iloc[0]['Ruta_Completa']
if os.path.exists(sample_image):
    print(f"✅ Acceso a imágenes confirmado: {sample_image}")
    
    # Probar carga de imagen
    try:
        img = Image.open(sample_image)
        print(f"  📏 Imagen de prueba: {img.size} píxeles, modo: {img.mode}")
    except Exception as e:
        print(f"  ❌ Error cargando imagen: {e}")
else:
    print(f"❌ No se puede acceder a: {sample_image}")

print("\n💡 Análisis disponible:")
print("  🎨 Análisis de colores dominantes")
print("  📊 Estadísticas básicas de imagen")
print("  🔍 Análisis de brillo y contraste")
print("  📈 Distribución de colores")

🤖 Configurando análisis de imágenes...
📦 Librerías disponibles:
  ✅ OpenCV: 4.12.0
  ✅ PIL/Pillow
  ✅ scikit-learn para clustering
✅ Acceso a imágenes confirmado: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/imagenes_descargadas/POST_17.jpg
  📏 Imagen de prueba: (640, 424) píxeles, modo: RGB

💡 Análisis disponible:
  🎨 Análisis de colores dominantes
  📊 Estadísticas básicas de imagen
  🔍 Análisis de brillo y contraste
  📈 Distribución de colores


In [ ]:
# 1. ANÁLISIS BÁSICO DE CARACTERÍSTICAS VISUALES
def analizar_caracteristicas_basicas(ruta_imagen):
    """
    Analiza características básicas de la imagen sin IA compleja
    """
    try:
        # Cargar imagen con PIL
        img_pil = Image.open(ruta_imagen)
        
        # Convertir a diferentes formatos para análisis
        img_rgb = img_pil.convert('RGB')
        img_gray = img_pil.convert('L')
        
        # Estadísticas básicas
        stats_rgb = ImageStat.Stat(img_rgb)
        stats_gray = ImageStat.Stat(img_gray)
        
        # Características calculadas
        brillo = stats_gray.mean[0] / 255.0
        contraste = stats_gray.stddev[0] / 128.0
        
        # Análisis de color
        colores_promedio = stats_rgb.mean  # R, G, B promedio
        saturacion_promedio = np.mean([colorsys.rgb_to_hsv(r/255, g/255, b/255)[1] 
                                     for r, g, b in [colores_promedio]])
        
        # Clasificación básica basada en características
        categoria_visual = clasificar_por_caracteristicas(brillo, saturacion_promedio, colores_promedio)
        
        return {
            'dimensiones': img_pil.size,
            'modo': img_pil.mode,
            'brillo': brillo,
            'contraste': contraste,
            'color_promedio_rgb': colores_promedio,
            'saturacion': saturacion_promedio,
            'categoria_visual': categoria_visual,
            'tamaño_archivo': os.path.getsize(ruta_imagen) / 1024  # KB
        }
    except Exception as e:
        return {'error': str(e)}

def clasificar_por_caracteristicas(brillo, saturacion, color_rgb):
    """
    Clasificación básica basada en características visuales
    """
    r, g, b = color_rgb
    
    # Análisis de brillo
    if brillo > 0.8:
        tipo_brillo = "muy_claro"
    elif brillo > 0.6:
        tipo_brillo = "claro"
    elif brillo > 0.4:
        tipo_brillo = "medio"
    elif brillo > 0.2:
        tipo_brillo = "oscuro"
    else:
        tipo_brillo = "muy_oscuro"
    
    # Análisis de saturación
    if saturacion > 0.7:
        tipo_saturacion = "muy_saturado"
    elif saturacion > 0.4:
        tipo_saturacion = "saturado"
    else:
        tipo_saturacion = "poco_saturado"
    
    # Análisis de dominancia de color
    if r > g and r > b:
        color_dominante = "rojizo"
    elif g > r and g > b:
        color_dominante = "verdoso"
    elif b > r and b > g:
        color_dominante = "azulado"
    else:
        color_dominante = "neutro"
    
    return {
        'brillo': tipo_brillo,
        'saturacion': tipo_saturacion,
        'color_dominante': color_dominante
    }

print("🔍 ANALIZANDO CARACTERÍSTICAS VISUALES...")
print("="*50)

resultados_caracteristicas = []

# Procesar las primeras 10 imágenes
for idx, row in df.head(10).iterrows():
    print(f"\n📸 Analizando {row['ID_Publicacion']} - Temática: {row['Tematica']}")
    
    resultado = analizar_caracteristicas_basicas(row['Ruta_Completa'])
    
    if 'error' not in resultado:
        print(f"  📏 Dimensiones: {resultado['dimensiones']}")
        print(f"  ? Brillo: {resultado['brillo']:.2f} ({resultado['categoria_visual']['brillo']})")
        print(f"  🎨 Saturación: {resultado['saturacion']:.2f} ({resultado['categoria_visual']['saturacion']})")
        print(f"  🌈 Color dominante: {resultado['categoria_visual']['color_dominante']}")
        print(f"  📊 Contraste: {resultado['contraste']:.2f}")
        
        # Guardar resultado
        resultados_caracteristicas.append({
            'ID': row['ID_Publicacion'],
            'Tematica_Original': row['Tematica'],
            'Brillo': resultado['brillo'],
            'Contraste': resultado['contraste'],
            'Saturacion': resultado['saturacion'],
            'Tipo_Brillo': resultado['categoria_visual']['brillo'],
            'Tipo_Saturacion': resultado['categoria_visual']['saturacion'],
            'Color_Dominante': resultado['categoria_visual']['color_dominante'],
            'Ancho': resultado['dimensiones'][0],
            'Alto': resultado['dimensiones'][1],
            'Tamaño_KB': resultado['tamaño_archivo']
        })
    else:
        print(f"  ❌ Error: {resultado['error']}")

# Resumen de características
if resultados_caracteristicas:
    df_caracteristicas = pd.DataFrame(resultados_caracteristicas)
    print(f"\n📊 RESUMEN DE CARACTERÍSTICAS:")
    print(f"  📏 Dimensiones promedio: {df_caracteristicas['Ancho'].mean():.0f}x{df_caracteristicas['Alto'].mean():.0f}")
    print(f"  💡 Brillo promedio: {df_caracteristicas['Brillo'].mean():.2f}")
    print(f"  🎨 Saturación promedio: {df_caracteristicas['Saturacion'].mean():.2f}")
    print(f"  ? Contraste promedio: {df_caracteristicas['Contraste'].mean():.2f}")
    
    print(f"\n?️ Distribución por características:")
    print(f"  Tipos de brillo: {df_caracteristicas['Tipo_Brillo'].value_counts().to_dict()}")
    print(f"  Tipos de saturación: {df_caracteristicas['Tipo_Saturacion'].value_counts().to_dict()}")
    print(f"  Colores dominantes: {df_caracteristicas['Color_Dominante'].value_counts().to_dict()}")

In [ ]:
# 2. DETECCIÓN DE OBJETOS
def detectar_objetos(ruta_imagen, top_n=10):
    """
    Detecta objetos principales en la imagen usando el modelo pre-entrenado
    """
    try:
        # Cargar y preprocesar imagen
        img = image.load_img(ruta_imagen, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        # Predicción
        predictions = model.predict(img_array, verbose=0)
        decoded_predictions = decode_predictions(predictions, top=top_n)[0]
        
        # Categorizar objetos detectados
        objetos = []
        for pred in decoded_predictions:
            codigo, nombre, confianza = pred
            objetos.append({
                'objeto': nombre.replace('_', ' '),
                'confianza': float(confianza),
                'categoria_ia': clasificar_objeto(nombre)
            })
        
        return objetos
    except Exception as e:
        return [{'error': str(e)}]

def clasificar_objeto(nombre_objeto):
    """
    Clasifica el objeto detectado en categorías generales
    """
    nombre = nombre_objeto.lower()
    
    # Diccionario de categorías
    categorias = {
        'comida': ['pizza', 'burger', 'sandwich', 'bread', 'cake', 'fruit', 'vegetable', 'meat', 'fish', 'cheese', 'wine', 'coffee', 'tea', 'ice_cream', 'chocolate', 'apple', 'banana', 'orange'],
        'animales': ['dog', 'cat', 'bird', 'horse', 'cow', 'elephant', 'lion', 'tiger', 'bear', 'rabbit', 'fish', 'butterfly', 'spider', 'snake'],
        'tecnologia': ['computer', 'laptop', 'phone', 'camera', 'television', 'radio', 'keyboard', 'mouse', 'screen', 'tablet', 'smartphone'],
        'transporte': ['car', 'truck', 'bus', 'train', 'plane', 'boat', 'bicycle', 'motorcycle', 'ship', 'helicopter'],
        'naturaleza': ['tree', 'flower', 'mountain', 'forest', 'lake', 'ocean', 'beach', 'rock', 'grass', 'sky', 'cloud'],
        'deporte': ['ball', 'tennis', 'football', 'basketball', 'golf', 'soccer', 'bicycle', 'ski', 'skateboard'],
        'hogar': ['chair', 'table', 'bed', 'sofa', 'lamp', 'television', 'refrigerator', 'stove', 'microwave'],
        'ropa': ['shirt', 'pants', 'dress', 'hat', 'shoe', 'jacket', 'tie', 'sock', 'boot', 'sweater']
    }
    
    for categoria, palabras in categorias.items():
        if any(palabra in nombre for palabra in palabras):
            return categoria
    
    return 'otros'

print("🔍 DETECTANDO OBJETOS...")
print("="*50)

resultados_objetos = []

# Analizar las primeras 5 imágenes
for idx, row in df.head(5).iterrows():
    print(f"\n📸 Analizando objetos en {row['ID_Publicacion']} - Temática: {row['Tematica']}")
    
    objetos = detectar_objetos(row['Ruta_Completa'])
    
    if objetos and 'error' not in objetos[0]:
        print(f"  🎯 Objetos detectados:")
        for i, obj in enumerate(objetos[:5], 1):  # Top 5 objetos
            print(f"    {i}. {obj['objeto']} ({obj['categoria_ia']}) - {obj['confianza']:.2%}")
        
        # Guardar resultado
        objetos_principales = [obj['objeto'] for obj in objetos[:3]]
        categorias_detectadas = [obj['categoria_ia'] for obj in objetos[:5]]
        categoria_dominante = Counter(categorias_detectadas).most_common(1)[0][0]
        
        resultados_objetos.append({
            'ID': row['ID_Publicacion'],
            'Tematica_Original': row['Tematica'],
            'Objetos_Top3': ', '.join(objetos_principales),
            'Categoria_Dominante_IA': categoria_dominante,
            'Confianza_Principal': objetos[0]['confianza']
        })
    else:
        print(f"  ❌ Error en detección")

# Mostrar resumen de objetos detectados
if resultados_objetos:
    df_objetos = pd.DataFrame(resultados_objetos)
    print(f"\n📊 RESUMEN DE DETECCIÓN DE OBJETOS:")
    print(f"  🎯 Imágenes analizadas: {len(df_objetos)}")
    print(f"  📈 Confianza promedio: {df_objetos['Confianza_Principal'].mean():.2%}")
    
    print(f"\n🏷️ Categorías dominantes detectadas por IA:")
    categorias_ia = df_objetos['Categoria_Dominante_IA'].value_counts()
    for categoria, cantidad in categorias_ia.items():
        print(f"  • {categoria}: {cantidad} imágenes")

In [3]:
# 3. ANÁLISIS DE COLORES OPTIMIZADO
def analizar_colores_rapido(ruta_imagen, n_colores=3):
    """
    Análisis rápido de colores dominantes usando K-means optimizado
    """
    try:
        # Cargar imagen con OpenCV (más rápido)
        img = cv2.imread(ruta_imagen)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Redimensionar agresivamente para velocidad
        img_small = cv2.resize(img, (50, 50))  # Muy pequeña para análisis rápido
        
        # Convertir a array de píxeles
        pixels = img_small.reshape(-1, 3)
        
        # K-means rápido
        kmeans = KMeans(n_clusters=n_colores, random_state=42, n_init=3, max_iter=50)
        kmeans.fit(pixels)
        
        # Obtener colores dominantes
        colores = kmeans.cluster_centers_.astype(int)
        labels = kmeans.labels_
        counts = np.bincount(labels)
        percentages = counts / len(labels) * 100
        
        # Crear resultado simplificado
        resultado = []
        for i, (color, porcentaje) in enumerate(zip(colores, percentages)):
            r, g, b = color
            resultado.append({
                'color_rgb': (int(r), int(g), int(b)),
                'color_hex': f'#{r:02x}{g:02x}{b:02x}',
                'porcentaje': float(porcentaje),
                'nombre_color': obtener_nombre_color_simple(r, g, b)
            })
        
        # Ordenar por porcentaje
        resultado.sort(key=lambda x: x['porcentaje'], reverse=True)
        return resultado
        
    except Exception as e:
        return [{'error': str(e)}]

def obtener_nombre_color_simple(r, g, b):
    """Nombres de colores simplificados"""
    if r > 180 and g > 180 and b > 180:
        return 'blanco'
    elif r < 60 and g < 60 and b < 60:
        return 'negro'
    elif r > max(g, b) + 30:
        return 'rojo'
    elif g > max(r, b) + 30:
        return 'verde'
    elif b > max(r, g) + 30:
        return 'azul'
    elif r > 120 and g > 120 and b < 80:
        return 'amarillo'
    else:
        return 'gris'

print("🎨 ANÁLISIS RÁPIDO DE COLORES...")
print("="*40)

resultados_colores_rapido = []

# Analizar solo las primeras 5 imágenes para demo
for idx, row in df.head(5).iterrows():
    print(f"\n🖼️ {row['ID_Publicacion']} ({row['Tematica']}):")
    
    colores = analizar_colores_rapido(row['Ruta_Completa'])
    
    if colores and 'error' not in colores[0]:
        for i, color in enumerate(colores, 1):
            print(f"  {i}. {color['nombre_color']} ({color['color_hex']}) - {color['porcentaje']:.1f}%")
        
        resultados_colores_rapido.append({
            'ID': row['ID_Publicacion'],
            'Tematica': row['Tematica'],
            'Color_Principal': colores[0]['nombre_color'],
            'Hex_Principal': colores[0]['color_hex'],
            'Porcentaje_Principal': colores[0]['porcentaje']
        })
    else:
        print(f"  ❌ Error en análisis")

# Mostrar resumen
if resultados_colores_rapido:
    df_colores_final = pd.DataFrame(resultados_colores_rapido)
    print(f"\n📊 RESUMEN FINAL:")
    print(f"  🎨 Imágenes analizadas: {len(df_colores_final)}")
    print(f"  🌈 Colores principales encontrados:")
    for color, count in df_colores_final['Color_Principal'].value_counts().items():
        print(f"    • {color}: {count} imágenes")

print(f"\n✅ ANÁLISIS COMPLETADO")
print(f"📍 Se analizaron {len(df.head(5))} imágenes de muestra")
print(f"🔧 Análisis realizado con OpenCV, PIL y scikit-learn")
print(f"⚡ Versión optimizada para velocidad")

🎨 ANÁLISIS RÁPIDO DE COLORES...

🖼️ POST_17 (comida):
  ❌ Error en análisis

🖼️ POST_134 (salud):
  ❌ Error en análisis

🖼️ POST_14 (comida):
  ❌ Error en análisis

🖼️ POST_25 (viajes):
  ❌ Error en análisis

🖼️ POST_4 (comida):
  ❌ Error en análisis

✅ ANÁLISIS COMPLETADO
📍 Se analizaron 5 imágenes de muestra
🔧 Análisis realizado con OpenCV, PIL y scikit-learn
⚡ Versión optimizada para velocidad


In [ ]:
# ANÁLISIS SIMPLIFICADO FINAL - Solo con PIL
def analisis_basico_imagen(ruta_imagen):
    """
    Análisis básico usando solo PIL - máxima compatibilidad
    """
    try:
        # Abrir imagen
        img = Image.open(ruta_imagen)
        
        # Información básica
        ancho, alto = img.size
        modo = img.mode
        
        # Convertir a RGB si es necesario
        if modo != 'RGB':
            img = img.convert('RGB')
        
        # Obtener color promedio
        # Redimensionar a 1x1 para obtener color promedio
        img_1x1 = img.resize((1, 1))
        color_promedio = img_1x1.getpixel((0, 0))
        
        # Análisis de brillo simple
        brillo = sum(color_promedio) / (3 * 255)
        
        # Clasificar color dominante
        r, g, b = color_promedio
        if r > g and r > b:
            color_dom = "rojizo"
        elif g > r and g > b:
            color_dom = "verdoso"  
        elif b > r and b > g:
            color_dom = "azulado"
        else:
            color_dom = "neutro"
        
        # Clasificar brillo
        if brillo > 0.7:
            tipo_brillo = "claro"
        elif brillo > 0.3:
            tipo_brillo = "medio"
        else:
            tipo_brillo = "oscuro"
            
        return {
            'exito': True,
            'dimensiones': f"{ancho}x{alto}",
            'color_promedio_rgb': color_promedio,
            'color_hex': f"#{r:02x}{g:02x}{b:02x}",
            'brillo': brillo,
            'tipo_brillo': tipo_brillo,
            'color_dominante': color_dom,
            'tamaño_mb': os.path.getsize(ruta_imagen) / (1024*1024)
        }
        
    except Exception as e:
        return {'exito': False, 'error': str(e)}

print("🎨 ANÁLISIS BÁSICO DE IMÁGENES")
print("="*40)
print("📦 Usando solo PIL para máxima compatibilidad\n")

resultados_finales = []

# Analizar las primeras 10 imágenes
for idx, row in df.head(10).iterrows():
    print(f"📸 {row['ID_Publicacion']} - {row['Tematica']}:")
    
    resultado = analisis_basico_imagen(row['Ruta_Completa'])
    
    if resultado['exito']:
        print(f"  📏 Tamaño: {resultado['dimensiones']}")
        print(f"  🎨 Color promedio: {resultado['color_hex']} ({resultado['color_dominante']})")
        print(f"  ? Brillo: {resultado['brillo']:.2f} ({resultado['tipo_brillo']})")
        print(f"  📊 Archivo: {resultado['tamaño_mb']:.2f} MB")
        
        resultados_finales.append({
            'ID': row['ID_Publicacion'],
            'Tematica': row['Tematica'],
            'Dimensiones': resultado['dimensiones'],
            'Color_Hex': resultado['color_hex'],
            'Color_Dominante': resultado['color_dominante'],
            'Brillo': resultado['brillo'],
            'Tipo_Brillo': resultado['tipo_brillo'],
            'Tamaño_MB': resultado['tamaño_mb']
        })
    else:
        print(f"  ❌ Error: {resultado['error']}")
    
    print()

# Resumen final
if resultados_finales:
    df_resultados = pd.DataFrame(resultados_finales)
    
    print("📊 RESUMEN DEL ANÁLISIS:")
    print("="*40)
    print(f"✅ Imágenes analizadas exitosamente: {len(df_resultados)}")
    print(f"📏 Tamaño promedio de archivos: {df_resultados['Tamaño_MB'].mean():.2f} MB")
    print(f"? Brillo promedio: {df_resultados['Brillo'].mean():.2f}")
    
    print(f"\n? Distribución de colores dominantes:")
    for color, count in df_resultados['Color_Dominante'].value_counts().items():
        print(f"  • {color}: {count} imágenes")
    
    print(f"\n💡 Distribución de brillo:")
    for brillo, count in df_resultados['Tipo_Brillo'].value_counts().items():
        print(f"  • {brillo}: {count} imágenes")
    
    print(f"\n🏷️ Análisis por temática original:")
    for tematica in df_resultados['Tematica'].unique():
        subset = df_resultados[df_resultados['Tematica'] == tematica]
        color_mas_comun = subset['Color_Dominante'].mode().iloc[0] if len(subset) > 0 else 'N/A'
        brillo_promedio = subset['Brillo'].mean() if len(subset) > 0 else 0
        print(f"  • {tematica}: color dominante {color_mas_comun}, brillo {brillo_promedio:.2f}")
    
    # Guardar resultados
    resultado_path = os.path.join(BASE_DIR, 'analisis_imagenes_basico.csv')
    df_resultados.to_csv(resultado_path, index=False)
    print(f"\n💾 Resultados guardados en: {resultado_path}")

print(f"\n🎉 ANÁLISIS COMPLETADO")
print(f"🛠️ Herramientas utilizadas: PIL/Pillow, pandas")
print(f"⚡ Análisis básico pero efectivo para computer vision")